# GTEx model building with PLIER2 and PLIER

# PLIER2

## Load libraries

In [2]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

# 3. Install PLIER2 (mchikina/PLIER2) if not already installed
if (!requireNamespace("PLIER2", quietly = TRUE)) {
    REPO_PATH <- "/home/msubirana/Documents/pivlab/PLIER2"  # adjust
    remotes::install_local(REPO_PATH, force = TRUE, dependencies = FALSE)
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER)
library(PLIER2)

source(here("config.R"))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/plier2-analyses

Loading required package: RColorBrewer

Loading required package: gplots


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: pheatmap

Loading required package: qvalue


Attaching package: ‘PLIER2’


The following objects are masked from ‘package:PLIER’:

    num.pc, projectPLIER




## Output directory

In [3]:
output_data_dir <- config$GTEx$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

## Download GTEx 

In [4]:
url <- config$GTEx$URL
dest_dir <-  config$GTEx$DATASET_FOLDER
dest_gz  <- file.path(dest_dir, basename(url))

if (!file.exists(dest_gz)) {
  dir.create(dest_dir, recursive = TRUE, showWarnings = FALSE)
  download.file(url, dest_gz, mode = "wb")
  message("Downloaded to: ", dest_gz)
} else {
  message("File already exists, skipping download.")
}

File already exists, skipping download.



## Preprocess GTEx data

In [10]:
exprs_path  <- file.path(config$GTEx$DATASET_FOLDER, 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz')
output_file <- config$GTEx$DATASET_FILE

if (!file.exists(output_file)) {
  dir.create(dirname(output_file), recursive = TRUE, showWarnings = FALSE)
  exprs_data <- read.table(exprs_path, header = TRUE, sep = "\t", skip = 2, check.names = FALSE)
  saveRDS(exprs_data, config$GTEx$DATASET_FILE)
  message("File successfully written to: ", config$GTEx$DATASET_FILE)
} else {
  message("Output file already exists. Skipping.")
}

# Aggregate in-place by 'description'
gtex <- readRDS(here(config$GTEx$DATASET_FILE))
gtex <- as.data.table(gtex)
aggregated_gtex <- gtex[, lapply(.SD, sum), by = Description, .SDcols = is.numeric]

genes <- aggregated_gtex$Description
samples <- colnames(aggregated_gtex[, -1])
data_mat <- as.matrix(aggregated_gtex[, -1])

Output file already exists. Skipping.



In [ ]:
# Create the FBM
fbm_file <- file.path(output_data_dir, "FBMgtex")
gtexFBM <- FBM(nrow = nrow(data_mat), ncol = ncol(data_mat), backingfile = fbm_file, create_bk = T)

# Populate it with data
block_size <- config$GENERAL$CHUNK_SIZE
n_blocks <- ceiling(nrow(aggregated_gtex) / block_size)

for (i in 1:n_blocks) {
  start_row <- (i-1) * block_size + 1
  end_row <- min(i * block_size, nrow(data_mat))
  
  gtexFBM[start_row:end_row, ] <- as.matrix(data_mat[start_row:end_row, ])
}

# Preprocess and z‑score FBM
prep_gtex <- preprocessPLIER2FBM(
  fbm        = gtexFBM,
  mean_cutoff= config$GTEx$GENES_MEAN_CUTOFF,
  var_cutoff = config$GTEx$GENES_VAR_CUTOFF
)

gtex_fbm_filt <- prep_gtex$fbm_filtered
gtex_rowStats <- prep_gtex$rowStats
zscorePLIER2FBM(gtex_fbm_filt, gtex_rowStats)
gtex_genes <- genes[prep_gtex$kept_rows]

Output file already exists. Skipping.

Applying log2 transformation

No NA values found

Applying Z-score transformation



In [8]:
exprs_path  <- file.path(config$GTEx$DATASET_FOLDER, 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz')
output_file <- config$GTEx$DATASET_FILE

if (!file.exists(output_file)) {
  dir.create(dirname(output_file), recursive = TRUE, showWarnings = FALSE)
  exprs_data <- read.table(exprs_path, header = TRUE, sep = "\t", skip = 2, check.names = FALSE)
  saveRDS(exprs_data, config$GTEx$DATASET_FILE)
  message("File successfully written to: ", config$GTEx$DATASET_FILE)
} else {
  message("Output file already exists. Skipping.")
}

# Aggregate in-place by 'description'
gtex <- readRDS(here(config$GTEx$DATASET_FILE))
gtex <- as.data.table(gtex)
aggregated_gtex <- gtex[, lapply(.SD, sum), by = Description, .SDcols = is.numeric]

genes <- aggregated_gtex$Description
samples <- colnames(aggregated_gtex[, -1])

Output file already exists. Skipping.



In [9]:
saveRDS(samples, file = file.path(output_data_dir, "gtex_samples.rds"))

In [15]:
saveRDS(gtex_genes, file = file.path(output_data_dir, "gtex_genes.rds"))

In [9]:
saveRDS(gtex_fbm_filt, file = file.path(output_data_dir, "gtex_fbm_filt.rds"))

## SVD computation and SVD K estimation

In [10]:
g_fb <- nrow(gtex_fbm_filt)
samples_fb <- ncol(gtex_fbm_filt)
SVD_K_gtex  <- min(g_fb, samples_fb) - 1

message("Using SVD K = ", SVD_K_gtex)

set.seed(config$GTEx$RANDOM_SVD_SEED)

gtex_svdRes <- big_randomSVD(gtex_fbm_filt, k = SVD_K_gtex)

Using SVD K = 17381



In [11]:
saveRDS(gtex_svdRes, file = file.path(output_data_dir, "gtex_svdRes.rds"))

## Estimate K for PLIER

In [12]:
PLIER_K_gtex <- num.pc(list(d = gtex_svdRes$d))
message("Inferred PLIER K = ", PLIER_K_gtex)

Inferred PLIER K = 206



In [13]:
saveRDS(PLIER_K_gtex, file = file.path(output_data_dir, "PLIER_K_gtex.rds"))

## PLIERbase initialization

In [14]:
gtex_baseRes <- PLIERbase(
  Y      = gtex_fbm_filt,
  k      = PLIER_K_gtex,
  svdres = gtex_svdRes,
  trace  = TRUE
)

****



[1] "L1 is set to 67.1332478272621"
[1] "L2 is set to 201.399743481786"
Progress 49 / 200 | Bdiff=0.000434, minCor=0.990968Converged at iteration= 49 | Bdiff=0.000434,  tol=0.000500     

In [15]:
saveRDS(gtex_baseRes, file = file.path(output_data_dir, "gtex_PLIER2_baseRes.rds"))

## Prepare pathway priors

In [41]:
gtex_gmtList <- list(
  GTEx_Tissues = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023"),
  BP            = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2025"),
  MSigDB        = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=MSigDB_Hallmark_2020")
)

gtex_pathMat <- gmtListToSparseMat(gtex_gmtList)
gtex_matched <- getMatchedPathwayMat(gtex_pathMat, gtex_genes)
gtex_chatObj <- getChat(gtex_matched)

there are 13567 genes in the intersction between data and prior

Removing 2020 pathways

Presolving using dot method

done



In [ ]:
gtex_gmtList <- list(
  GTEx_Tissues = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023"),
  BP            = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2025"),
  MSigDB        = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=MSigDB_Hallmark_2020")
)

gtex_pathMat <- gmtListToSparseMat(gtex_gmtList)
gtex_matched <- getMatchedPathwayMat(gtex_pathMat, gtex_genes)
gtex_chatObj <- getChat(gtex_matched)

there are 13567 genes in the intersction between data and prior

Removing 2020 pathways

Presolving using dot method

done



## PLIERfull

In [ ]:
gtex_fullRes <- PLIERfull(
  Y                 = gtex_fbm_filt,
  priorMat          = as.matrix(gtex_matched),
  svdres            = gtex_svdRes,
  plier.base.result = gtex_baseRes,
  Chat              = gtex_chatObj,
  k                 = PLIER_K_gtex,
  doCrossval        = TRUE,
  trace             = TRUE,
  max.U.updates= config$GTEx$MAX_U_UPDATES
)

**PLIER v2 **

using provided PLIERbase result

L1=67.1332478272621; L2=201.399743481786



Progress 80 / 350 | Bdiff=0.000171 Number of annotated columns is 190, Number of annotated columns is 190, Number of annotated columns is 190, Number of annotated columns is 190, Number of annotated columns is 190, Number of annotated columns is 188, Number of annotated columns is 188, Number of annotated columns is 188, Number of annotated columns is 189, Number of annotated columns is 189, Number of annotated columns is 186, Number of annotated columns is 186, Number of annotated columns is 186, Number of annotated columns is 187, Number of annotated columns is 187, Number of annotated columns is 187, Number of annotated columns is 187, Number of annotated columns is 187, Number of annotated columns is 187, Number of annotated columns is 188, Number of annotated columns is 186, Number of annotated columns is 186, Number of annotated columns is 186, Number of annotated columns is 187, Number of annotated columns is 188, Number of annotated columns is 188, Number of annotated columns i

Bdiff is not decreasing



Progress 81 / 350 | Bdiff=0.00017189

Bdiff is not decreasing



...

Bdiff is not decreasing



Progress 86 / 350 | Bdiff=0.00018789

Bdiff is not decreasing



Converged at 108 / 350 | Bdiff=0.000132, minCor=0.993962d columns is 189, Number of annotated columns is 190, Number of annotated columns is 190, Number of annotated columns is 190, Number of annotated columns is 190, Number of annotated columns is 190, Number of annotated columns is 190


Updating Z for CV

crossValidation

There are 162  LVs with AUC>0.70

There are 129  LVs with AUC>0.90



In [43]:
saveRDS(gtex_fullRes, file = file.path(output_data_dir, "gtex_PLIER2.rds"))

## Fix colnames and rownames

In [4]:
gtex_fullRes <- readRDS(file.path(output_data_dir, "gtex_PLIER2.rds"))

In [11]:
length(samples)

[1] 17382

In [12]:
head(gtex_fullRes$B)
dim(gtex_fullRes$B)

LV1,0.059449495,-0.157660594,0.41945902,0.32562436,0.05399605,-0.12757005,0.10439109,-0.15271585,-0.103412405,-0.08526571,⋯,0.116236290,-0.10269113,0.10244273,-0.10777651,-0.04678458,0.21613528,-0.239800891,0.70461407,-0.20576332,0.05461336
LV2,0.024647632,-0.069506978,-0.03927263,0.06580254,0.02507455,0.07497131,0.11967017,0.05680829,0.074638258,0.05380043,⋯,-0.036538189,-0.08268311,-0.02050416,-0.03650432,-0.04460744,-0.04375957,-0.004767715,-0.08815896,-0.04758426,0.00358794
LV3,0.001745996,-0.164543763,0.26281525,0.09615976,0.28775519,0.87790287,-0.01135481,0.12534965,0.293755692,-0.11067616,⋯,-0.008751971,-0.05602059,0.42797920,0.37534993,-0.04101079,-0.00912129,0.040197562,-0.06202747,-0.14657807,0.15536284
LV4,-0.030103405,-0.007088976,-0.02412439,-0.03070101,-0.01030226,0.04757672,-0.02888953,-0.05948635,-0.058873693,-0.03254389,⋯,-0.027701068,-0.05825118,-0.02772065,-0.03185123,-0.04991498,-0.11033416,-0.050303455,-0.06259047,-0.01519923,-0.04838368
LV5,-0.266252622,-0.332131300,-0.39778680,-0.25790455,0.16385956,-0.23529447,-0.23104312,0.12040320,-0.183396378,-0.14096004,⋯,0.550067001,0.21607662,0.13027950,0.18418594,0.02035761,-0.17362448,0.184564640,-0.30362634,0.07145606,-0.02883381
LV6,0.105200849,-0.017466987,0.08719982,0.04069410,-0.12607696,-0.09013901,0.09270272,-0.01672574,-0.002047877,-0.07069843,⋯,-0.130224455,-0.08737340,0.04020525,-0.12147980,-0.04907285,0.01885870,-0.141264247,-0.02010885,-0.10966572,-0.01358866


[1]   206 17382

In [17]:
colnames(gtex_fullRes$B) <- samples

In [18]:
head(gtex_fullRes$B)
dim(gtex_fullRes$B)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
LV1,0.059449495,-0.157660594,0.41945902,0.32562436,0.05399605,-0.12757005,0.10439109,-0.15271585,-0.103412405,-0.08526571,⋯,0.116236290,-0.10269113,0.10244273,-0.10777651,-0.04678458,0.21613528,-0.239800891,0.70461407,-0.20576332,0.05461336
LV2,0.024647632,-0.069506978,-0.03927263,0.06580254,0.02507455,0.07497131,0.11967017,0.05680829,0.074638258,0.05380043,⋯,-0.036538189,-0.08268311,-0.02050416,-0.03650432,-0.04460744,-0.04375957,-0.004767715,-0.08815896,-0.04758426,0.00358794
LV3,0.001745996,-0.164543763,0.26281525,0.09615976,0.28775519,0.87790287,-0.01135481,0.12534965,0.293755692,-0.11067616,⋯,-0.008751971,-0.05602059,0.42797920,0.37534993,-0.04101079,-0.00912129,0.040197562,-0.06202747,-0.14657807,0.15536284
LV4,-0.030103405,-0.007088976,-0.02412439,-0.03070101,-0.01030226,0.04757672,-0.02888953,-0.05948635,-0.058873693,-0.03254389,⋯,-0.027701068,-0.05825118,-0.02772065,-0.03185123,-0.04991498,-0.11033416,-0.050303455,-0.06259047,-0.01519923,-0.04838368
LV5,-0.266252622,-0.332131300,-0.39778680,-0.25790455,0.16385956,-0.23529447,-0.23104312,0.12040320,-0.183396378,-0.14096004,⋯,0.550067001,0.21607662,0.13027950,0.18418594,0.02035761,-0.17362448,0.184564640,-0.30362634,0.07145606,-0.02883381
LV6,0.105200849,-0.017466987,0.08719982,0.04069410,-0.12607696,-0.09013901,0.09270272,-0.01672574,-0.002047877,-0.07069843,⋯,-0.130224455,-0.08737340,0.04020525,-0.12147980,-0.04907285,0.01885870,-0.141264247,-0.02010885,-0.10966572,-0.01358866


[1]   206 17382

In [20]:
head(gtex_fullRes$Z)
dim(gtex_fullRes$Z)

,LV 1,LV 2,LV 3,LV 4,LV 5,LV 6,LV 7,LV 8,LV 9,LV 10,⋯,LV 197,LV 198,LV 199,LV 200,LV 201,LV 202,LV 203,LV 204,LV 205,LV 206
WASH7P,0,0,0,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.0000000,0,0,0.2660610,0.0000000,0.0000000
RP11-34P13.15,0,0,0,0,0,0.3892596,0.1716437,0,0,0,⋯,0,0,0,0,0.3115469,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.16,0,0,0,0,0,0.3688137,0.1549155,0,0,0,⋯,0,0,0,0,0.3386183,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.18,0,0,0,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.2069137,0,0,0.1896527,0.0000000,0.0000000
AP006222.2,0,0,0,0,0,0.0000000,0.1948309,0,0,0,⋯,0,0,0,0,0.3537102,0,0,0.1384222,0.1494011,0.0000000
MTND1P23,0,0,0,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.4091260,0,0,0.0000000,0.0000000,0.4541037


[1] 21613   206

In [21]:
colnames(gtex_fullRes$Z) <- paste0('LV', seq_len(ncol(gtex_fullRes$Z)))

In [22]:
head(gtex_fullRes$Z)
dim(gtex_fullRes$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV197,LV198,LV199,LV200,LV201,LV202,LV203,LV204,LV205,LV206
WASH7P,0,0,0,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.0000000,0,0,0.2660610,0.0000000,0.0000000
RP11-34P13.15,0,0,0,0,0,0.3892596,0.1716437,0,0,0,⋯,0,0,0,0,0.3115469,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.16,0,0,0,0,0,0.3688137,0.1549155,0,0,0,⋯,0,0,0,0,0.3386183,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.18,0,0,0,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.2069137,0,0,0.1896527,0.0000000,0.0000000
AP006222.2,0,0,0,0,0,0.0000000,0.1948309,0,0,0,⋯,0,0,0,0,0.3537102,0,0,0.1384222,0.1494011,0.0000000
MTND1P23,0,0,0,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.4091260,0,0,0.0000000,0.0000000,0.4541037


[1] 21613   206

In [23]:
head(gtex_fullRes$summary)

,pathway,LV index,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Artery - Aorta Female 60-69 Up,1,0.9909541,2.203697e-15,5.574661e-14
2,Artery - Aorta Male 50-59 Up,1,0.9953669,6.674579e-15,9.386126e-14
3,Artery - Aorta Female 20-29 Up,1,0.9910400,2.179744e-15,5.574661e-14
4,Artery - Aorta Female 50-59 Up,1,0.9880541,3.183981e-15,6.668281e-14
5,Artery - Aorta Male 30-39 Up,1,0.9941387,1.467544e-15,4.773775e-14
6,Artery - Aorta Female 30-39 Up,1,0.9903604,2.376553e-15,5.628678e-14


In [25]:
gtex_fullRes$summary <- gtex_fullRes$summary %>%
    dplyr::rename(LV = `LV index`)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

In [26]:
head(gtex_fullRes$summary)

,pathway,LV,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Artery - Aorta Female 60-69 Up,LV1,0.9909541,2.203697e-15,5.574661e-14
2,Artery - Aorta Male 50-59 Up,LV1,0.9953669,6.674579e-15,9.386126e-14
3,Artery - Aorta Female 20-29 Up,LV1,0.9910400,2.179744e-15,5.574661e-14
4,Artery - Aorta Female 50-59 Up,LV1,0.9880541,3.183981e-15,6.668281e-14
5,Artery - Aorta Male 30-39 Up,LV1,0.9941387,1.467544e-15,4.773775e-14
6,Artery - Aorta Female 30-39 Up,LV1,0.9903604,2.376553e-15,5.628678e-14


In [27]:
gtex_fullRes$summary  %>% 
dplyr::filter(FDR < 0.05) %>% 
dplyr::filter(AUC > 0.7)  %>% 
tail()

,pathway,LV,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1050,Translation (GO:0006412),LV202,0.8599546,4.551380e-17,5.120303e-15
1051,Oxidative Phosphorylation,LV202,0.8838041,4.267331e-17,5.120303e-15
1052,Adipose - Visceral (Omentum) Male 30-39 Up,LV203,0.7972713,9.732122e-07,1.469616e-06
1053,Adipose - Visceral (Omentum) Female 30-39 Up,LV203,0.7227635,1.810210e-04,2.322988e-04
1054,Adipose - Visceral (Omentum) Male 20-29 Up,LV203,0.7998422,7.930707e-07,1.207041e-06
1055,Fallopian Tube Female 40-49 Up,LV203,0.7646562,4.618164e-04,5.772706e-04


In [28]:
saveRDS(gtex_fullRes, file = file.path(output_data_dir, "gtex_PLIER2.rds"))

# PLIER

Run PLIER with the same inputs than PLIER2


In [53]:
gtex_plier = PLIER::PLIER(
    gtex_fbm_filt[], 
    as.matrix(gtex_matched), 
    svdres = gtex_svdRes, 
    Chat = as.matrix(gtex_chatObj), 
    doCrossval = TRUE, 
    max.iter = config$GTEx$PLIER_PARAMS$MAX_ITER,
    k = PLIER_K_gtex
  )

Removing 0 pathways with too few genes



[1] 201.3997
[1] "L2 is set to 201.399743481786"
[1] "L1 is set to 100.699871740893"


errorY (SVD based:best possible) = 0.3686

New L3 is 0.000804733010124613

New L3 is 0.000488095243523415

New L3 is 0.000430742540575688

New L3 is 0.000430742540575688

New L3 is 0.000430742540575688

New L3 is 0.000380128957869464

New L3 is 0.000430742540575688

New L3 is 0.000380128957869464

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

converged at  iteration 179 Bdiff is not decreasing

There are 132  LVs with AUC>0.70



In [54]:
saveRDS(gtex_plier, file = file.path(output_data_dir, "gtex_PLIER.rds"))

In [5]:
gtex_plier <- readRDS(file.path(output_data_dir, "gtex_PLIER.rds"))

In [6]:
colnames(gtex_plier$Z) <- paste0('LV', seq_len(ncol(gtex_plier$Z)))

In [9]:
head(gtex_plier$Z)
dim(gtex_plier$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV197,LV198,LV199,LV200,LV201,LV202,LV203,LV204,LV205,LV206
WASH7P,0.0000000,0.02499416,0.000000000,0.00000000,0.076692203,0.00000000,0.01748135,0.00000000,0.09224386,0.0039521771,⋯,0.03193649,0.19097628,0.03509836,1.0084783,0.0000000,0.0005897013,0.00000000,0.017723577,0.00000000,0.00000000
RP11-34P13.15,0.0000000,0.03599516,0.088766434,0.66692284,0.000000000,0.40973066,0.00000000,0.00000000,0.00000000,0.0000000000,⋯,0.00000000,0.00000000,0.17049253,0.2512257,0.2381905,0.0000000000,0.07925430,0.158438333,0.06908235,0.00000000
RP11-34P13.16,0.0000000,0.04031925,0.051044799,0.68430944,0.006468101,0.44774443,0.00000000,0.00000000,0.00000000,0.0006706749,⋯,0.00000000,0.00000000,0.17280849,0.2140185,0.2540909,0.0000000000,0.09339478,0.169480128,0.07519298,0.00000000
RP11-34P13.18,0.0000000,0.00000000,0.048702159,0.00000000,0.085199947,0.24553884,0.00000000,0.00000000,0.04624262,0.0000000000,⋯,0.03087897,0.00000000,0.00916668,0.8645280,0.1606547,0.0000000000,0.03617533,0.007241217,0.00000000,0.00000000
AP006222.2,0.0000000,0.00000000,0.005665622,0.06150562,0.000000000,0.01158631,0.00000000,0.00000000,0.00000000,0.0000000000,⋯,0.00000000,0.03828548,0.01798606,0.3520241,0.2906519,0.0000000000,0.00000000,0.000000000,0.09057921,0.05701279
MTND1P23,0.2806694,0.00000000,0.000000000,0.03020411,0.055460063,0.00000000,0.06821846,0.06779852,0.00000000,0.0000000000,⋯,0.02897674,0.29230921,0.00000000,0.0000000,0.1892596,0.0866651624,0.00000000,0.196224263,0.05855393,0.20566792


[1] 21613   206

In [8]:
gtex_plier$summary <- gtex_plier$summary %>%
    dplyr::rename(LV = `LV index`)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

In [9]:
head(gtex_plier$summary)

,pathway,LV,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Cells - EBV-transformed Lymphocytes Female 20-29 Up,LV1,0.9917422,7.193265e-12,2.380514e-11
2,Cells - EBV-transformed Lymphocytes Male 30-39 Up,LV1,0.9889461,2.023296e-12,7.484304e-12
3,Cells - EBV-transformed Lymphocytes Female 50-59 Up,LV1,0.9887520,2.063090e-12,7.588951e-12
4,Cells - EBV-transformed Lymphocytes Male 60-69 Up,LV1,0.9872147,2.406492e-12,8.754499e-12
5,Cells - EBV-transformed Lymphocytes Male 20-29 Up,LV1,0.9967393,9.200835e-13,3.674035e-12
6,Cells - EBV-transformed Lymphocytes Female 40-49 Up,LV1,0.9805983,4.643783e-12,1.601054e-11


In [13]:
head(gtex_plier$B)
dim(gtex_plier$B)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
"1,Cells - EBV-transformed Lymphocytes Male 40-49 Up",0.46617279,-0.160274908,0.73189638,0.03673337,-0.16036737,0.27011287,0.34155448,0.18709101,0.124303805,0.17580335,⋯,-0.23524018,-0.04565981,0.192526401,-0.07712182,0.18866958,0.49366655,0.083557804,0.34700683,-0.3813536716,0.23392498
"2,Colon - Transverse Male 70-79 Up",-0.05482741,0.031944407,-0.07119695,-0.03778371,0.07130929,-0.03862763,-0.10123493,-0.01607100,-0.094175281,-0.04041773,⋯,0.03768582,0.01823106,-0.086882211,0.19412630,0.28690821,-0.08885060,0.038956649,-0.06684843,0.0695568037,-0.04009313
LV 3,-0.18970150,0.084744719,-0.04421499,-0.12689688,-0.01891176,-0.03325522,-0.20798620,0.89332084,-0.003713821,0.04296174,⋯,-0.01377714,-0.08280761,0.009087268,0.09793773,0.05386236,-0.19782578,0.423212881,0.07848064,-0.0234689283,-0.05235816
"4,Whole Blood Female 50-59 Up",-0.16835215,-0.005646782,-0.14882507,-0.13028856,0.04456463,-0.14427548,-0.09085780,-0.08721934,-0.070949456,-0.13710315,⋯,0.02282449,-0.16052529,-0.080124805,-0.05021261,-0.11735479,-0.09932323,-0.088999635,-0.05632542,0.0002726316,-0.06148682
"5,Myc Targets V1",0.22784539,0.135854387,0.02489118,0.14955940,-0.35129585,0.06705728,0.04226745,0.15924572,0.305569910,0.14872014,⋯,-0.19613522,0.17144165,-0.187114670,-0.06161684,0.01157285,0.04681580,0.413042999,-0.09198360,0.2846712048,0.04444101
LV 6,-0.14773172,-0.090280367,-0.09726386,-0.17305484,-0.10486026,-0.16211997,-0.24268387,-0.18460777,-0.137521121,-0.15737880,⋯,0.05857534,0.05339165,0.067576095,0.03867730,0.09123170,0.09281796,0.009773036,0.06410904,0.0467131491,0.09168696


[1]   206 17382

In [12]:
colnames(gtex_plier$B) <- samples

In [14]:
saveRDS(gtex_plier, file = file.path(output_data_dir, "gtex_PLIER.rds"))